<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/temp_hosting/rel_strength.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import yfinance as yf
# Example stock data (replace with your actual data)
nifty50_url = "https://raw.githubusercontent.com/anirbanghoshsbi/data/main/ind_nifty50list.csv"
nifty50 = pd.read_csv(nifty50_url)
tickers = nifty50['Symbol'].apply(lambda x: x + ".NS").tolist()
# Bank Nifty Stocks
nifty_stocks = tickers
start_date = '2023-01-02'

def download_data(stock_list,start_date):
    data = yf.download(stock_list, start=start_date)
    data = data[['Adj Close']]
    return data

# Example benchmark index data (replace with your actual data)
benchmark_index = {
    'Index': ['MyIndex'],
    'Value': [2000]
}

# Create DataFrames
stock_df = download_data(nifty_stocks, start_date)
benchmark_df = yf.download('^nsei',start_date)

# Assuming stock_data is your DataFrame with MultiIndex columns

# Reset the index to flatten it
stock_df.reset_index(inplace=True)

# Reset the columns index to flatten the MultiIndex
stock_df.columns = stock_df.columns.get_level_values(1)

# Convert index to datetime
stock_df.columns.values[0]='Date'

# Calculate percentage change for each stock
percentage_change_data_90 = stock_df.set_index('Date').pct_change(90)*100
percentage_change_data_120 = stock_df.set_index('Date').pct_change(120)*100
percentage_change_data_150 = stock_df.set_index('Date').pct_change(150)*100
percentage_change_data_180 = stock_df.set_index('Date').pct_change(180)*100
# Create a new DataFrame to store the percentage change data
percentage_change_df_90 = pd.DataFrame(percentage_change_data_90)
percentage_change_df_120 = pd.DataFrame(percentage_change_data_120)
percentage_change_df_150 = pd.DataFrame(percentage_change_data_150)
percentage_change_df_180 = pd.DataFrame(percentage_change_data_180)




benchmark_df['Percentage Change_90'] = benchmark_df['Adj Close'].pct_change(90) * 100
benchmark_df['Percentage Change_120'] = benchmark_df['Adj Close'].pct_change(120) * 100
benchmark_df['Percentage Change_150'] = benchmark_df['Adj Close'].pct_change(150) * 100
benchmark_df['Percentage Change_180'] = benchmark_df['Adj Close'].pct_change(180) * 100

# Remove NaN values
stock_df.dropna(inplace=True)
benchmark_df.dropna(inplace=True)



[*********************100%%**********************]  50 of 50 completed
[*********************100%%**********************]  1 of 1 completed


In [71]:
merged_df = percentage_change_df_90.join(benchmark_df['Percentage Change_90'], how='inner')

In [72]:
covariance = merged_df.cov()['Percentage Change_90']
benchmark_variance = np.var(benchmark_df['Percentage Change_90'])
beta = covariance / benchmark_variance

In [73]:
# Get the last row of stock_df
last_row_90 = percentage_change_df_90.iloc[-1]  # or stock_df.tail(1)
last_row_120 = percentage_change_df_120.iloc[-1]
last_row_150 = percentage_change_df_150.iloc[-1]
last_row_180 = percentage_change_df_180.iloc[-1]
# Create a new DataFrame with the last row and the beta series
new_df = pd.DataFrame({
    'Percentage_change_90': last_row_90,
    'Percentage_change_120': last_row_120,
    'Percentage_change_150': last_row_150,
    'Percentage_change_180': last_row_180,
    'Beta': beta,
    'volatility':np.std(percentage_change_df_90),


})



In [74]:
# Reset the index of the new DataFrame
new_df = new_df.dropna().reset_index()


In [75]:
new_df['Relative_Strength_90']=new_df['Percentage_change_90']/benchmark_df['Percentage Change_90'].iloc[-1]
new_df['Relative_Strength_120']=new_df['Percentage_change_120']/benchmark_df['Percentage Change_120'].iloc[-1]
new_df['Relative_Strength_150']=new_df['Percentage_change_150']/benchmark_df['Percentage Change_150'].iloc[-1]
new_df['Relative_Strength_180']=new_df['Percentage_change_180']/benchmark_df['Percentage Change_180'].iloc[-1]

In [76]:
# Optional: Clustering
X= new_df[['Relative_Strength_90','Relative_Strength_120','Relative_Strength_150','Relative_Strength_180','volatility','Beta']].values
kmeans = KMeans(n_clusters=10)
kmeans.fit(X)
new_df['Cluster'] = kmeans.labels_

# Print clustering results
print(new_df[['index', 'Cluster']])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


            index  Cluster
0     ADANIENT.NS        7
1   ADANIPORTS.NS        2
2   APOLLOHOSP.NS        5
3   ASIANPAINT.NS        4
4     AXISBANK.NS        0
5   BAJAJ-AUTO.NS        9
6   BAJAJFINSV.NS        0
7   BAJFINANCE.NS        4
8   BHARTIARTL.NS        6
9         BPCL.NS        3
10   BRITANNIA.NS        0
11       CIPLA.NS        4
12   COALINDIA.NS        2
13    DIVISLAB.NS        4
14     DRREDDY.NS        5
15   EICHERMOT.NS        6
16      GRASIM.NS        0
17     HCLTECH.NS        5
18    HDFCBANK.NS        8
19    HDFCLIFE.NS        4
20  HEROMOTOCO.NS        1
21    HINDALCO.NS        1
22  HINDUNILVR.NS        8
23   ICICIBANK.NS        0
24  INDUSINDBK.NS        4
25        INFY.NS        4
26         ITC.NS        4
27    JSWSTEEL.NS        0
28   KOTAKBANK.NS        8
29          LT.NS        5
30        LTIM.NS        4
31         M&M.NS        1
32      MARUTI.NS        6
33   NESTLEIND.NS        0
34        NTPC.NS        1
35        ONGC.NS        9
3

Easy-to-Remember Cluster Performance Table:
Cluster	Current State	Examples & Notes


0	😐 Mixed	JSWSTEEL.NS - Growth, Others Stagnant


1	📈 Strong Growth	BHARTIARTL.NS, NTPC.NS, SBIN.NS


2	📉 Underperforming	ASIANPAINT.NS, HDFCLIFE.NS, KOTAKBANK.NS


3	🌱 Moderate Growth	APOLLOHOSP.NS, DRREDDY.NS, TATACONSUM.NS


4	😟 Challenges & Uncertainty	BAJFINANCE.NS, HDFCBANK.NS, INFY.NS


5	🚀 Outperforming	ADANIPORTS.NS, BPCL.NS, COALINDIA.NS


6	📈🚀 High Growth, High Beta	BAJAJ-AUTO.NS, ONGC.NS


7	👍 Positive Performance	CIPLA.NS, HEROMOTOCO.NS, SUNPHARMA.NS


8	📈🎢 Growth with Volatility	HCLTECH.NS, ICICIBANK.NS, WIPRO.NS


9	🚀🚀 High Growth, High Volatility	ADANIENT.NS - Potential for Big Gains/Losses

Analysis of Current Stock Clusters:
Based on the provided data, here's an interpretation of the current state of each cluster:


Cluster 0:
Mixed Performance: This cluster shows a mix of performance, with some stocks like JSWSTEEL.NS experiencing moderate growth while others like NESTLEIND.NS and TITAN.NS remaining relatively stagnant.
Potential Turnaround: The positive performance of JSWSTEEL.NS might indicate a potential turnaround or positive developments within its sector.


Cluster 1:
Growth & Stability: Stocks in this cluster are displaying strong growth, as seen with BHARTIARTL.NS, NTPC.NS, POWERGRID.NS, and SBIN.NS.
Positive Investor Sentiment: This could signify positive investor sentiment and potential future growth within these companies and their respective sectors.


Cluster 2:
Underperformance: This cluster currently consists of stocks experiencing negative returns, including ASIANPAINT.NS, HDFCLIFE.NS, HINDUNILVR.NS, KOTAKBANK.NS, LTIM.NS, and UPL.NS.
Potential Concerns: This might indicate challenges or negative developments within their sectors or specific company issues impacting investor confidence.


Cluster 3:
Moderate Growth: Stocks in this cluster are exhibiting moderate growth and positive returns, including APOLLOHOSP.NS, DRREDDY.NS, EICHERMOT.NS, GRASIM.NS, and TATACONSUM.NS.
Steady Performance: This suggests stable performance and potential for continued growth within these companies.


Cluster 4:
Challenges and Uncertainty: This cluster is facing challenges, with stocks like BAJAJFINSV.NS, BAJFINANCE.NS, HDFCBANK.NS, INDUSINDBK.NS, INFY.NS, ITC.NS, and ULTRACEMCO.NS experiencing negative returns.
Economic or Sector-Specific Issues: This could be due to broader economic concerns, sector-specific challenges, or company-specific issues impacting investor confidence.


Cluster 5:
Outperformance: This cluster is demonstrating strong performance, with ADANIPORTS.NS, BPCL.NS, and COALINDIA.NS showing significant positive returns.
Positive Developments: This suggests potential positive developments or increased investor interest within these companies and their sectors.


Cluster 6:
Strong Growth & High Beta: BAJAJ-AUTO.NS and ONGC.NS in this cluster are exhibiting high growth, aligning with their high beta characteristics.
Investor Confidence: This indicates strong investor confidence and potential for further growth in these companies.


Cluster 7:
Positive Performance: This cluster is performing well, with CIPLA.NS, HEROMOTOCO.NS, HINDALCO.NS, MARUTI.NS, RELIANCE.NS, and SUNPHARMA.NS showing positive returns.
Growth Potential: This suggests potential for continued growth and positive investor sentiment surrounding these companies.


Cluster 8:
Growth with Some Volatility: DIVISLAB.NS, HCLTECH.NS, ICICIBANK.NS, and WIPRO.NS in this cluster are experiencing growth but with a degree of volatility.
Potential for Fluctuation: This indicates potential for future price fluctuations and the need for careful monitoring of these stocks.


Cluster 9:
High Growth & Volatility: ADANIENT.NS is exhibiting high growth aligning with its high-risk, high-reward characteristics.


Potential for Significant Gains or Losses: Investors should be aware of the potential for significant gains but also substantial losses due to the inherent volatility of this stock.

***Cluster Analysis with Multi-Period Performance:***
Given the data with percentage change and relative strength over multiple periods (90, 120, 150, and 180 days), here's a breakdown of insights for each cluster:

Cluster 0:
Moderate Growth with Increasing Stability: Stocks exhibit moderate but consistent growth across all periods. Volatility seems to be decreasing, leading to more stability.
Potential for Steady Returns: This cluster may be attractive for investors seeking reliable returns with moderate growth potential.

Cluster 1:
High Growth & Momentum: Stocks demonstrate strong performance and increasing momentum across all periods. Volatility remains relatively consistent.
Potential for High Returns but with Risk: This cluster might be suitable for investors comfortable with higher risk for potentially higher returns.

Cluster 2:
Low Volatility & Consistent Growth: Stocks exhibit low volatility and steady growth throughout the periods.
Potential for Stable Returns with Lower Risk: This cluster could be ideal for investors seeking stable returns with lower risk and moderate growth.

Cluster 3:
Varied Performance & Volatility: Stocks show a mix of performance, with some experiencing consistent growth (DIVISLAB.NS) while others struggle (ASIANPAINT.NS). Volatility also varies among stocks.
Requires Careful Analysis: Investors need to analyze individual stocks within this cluster carefully due to the varied performance and risk profiles.

Cluster 4:
Moderate Growth & Decreasing Volatility: Stocks show overall positive performance with a trend of decreasing volatility, indicating increasing stability.
Potential for Improving Returns with Lower Risk: This cluster may be suitable for investors seeking a balance between growth and risk, as volatility seems to be diminishing.

Cluster 5:
Strong and Consistent Growth: Stocks demonstrate consistently high growth across all periods, indicating strong momentum and positive investor sentiment.
High Growth Potential but with Fluctuations: This cluster might be appropriate for investors seeking high-growth opportunities, but they should be prepared for potential price fluctuations.

Cluster 6:
High Volatility and Growth: Stocks experience significant price swings but also demonstrate strong growth potential.
High-Risk, High-Reward Potential: This cluster is suitable for investors with a high-risk tolerance and a long-term perspective, as the potential for high returns comes with significant volatility.

Cluster 7:
Strong Performance with Some Volatility: Stocks exhibit overall strong performance, but some experience fluctuations in growth and relative strength.
Growth Potential with Moderate Risk: This cluster may appeal to investors seeking growth opportunities with a moderate level of risk.

Cluster 8:
Varied Performance & Moderate Growth: Stocks show mixed performance with some experiencing consistent growth while others face challenges.
Requires Individual Stock Analysis: Careful analysis of individual stocks within this cluster is crucial due to the diverse performance and risk profiles.

Cluster 9:
Low Volatility & Moderate Growth: Stocks experience low volatility and steady growth throughout the periods.
Potential for Stable Returns with Lower Risk: This cluster could be suitable for investors seeking reliable returns with moderate growth and lower risk.

Easy-to-Remember Cluster Summary:
Cluster	Key Traits	Investor Suitability


0	📈 Moderate & Stable Growth	Steady Returns, Moderate Risk


1	🚀🚀 High Growth & Momentum	High Returns, Higher Risk


2	🌱 Low Volatility & Steady Growth	Stable Returns, Lower Risk


3	📈🎢 Mixed Performance & Volatility	Requires Careful Analysis


4	📈📉 Moderate Growth & Decreasing Volatility	Improving Returns, Lowering Risk


5	🚀 Consistent High Growth	High Growth Potential, Some Fluctuations


6	🎢🚀 High Volatility & Growth	High-Risk, High-Reward


7	📈 Strong Performance & Some Volatility	Growth Potential, Moderate Risk


8	📈😐 Varied Performance & Growth	Requires Individual Stock Analysis


9	🌱 Low Volatility & Moderate Growth	Stable Returns, Lower Risk

In [81]:
new_df=new_df[['index','Relative_Strength_90','Relative_Strength_120','Relative_Strength_150','Relative_Strength_180','volatility','Beta','Cluster']]
df_cluster_3 = new_df[new_df['Cluster'] ==8]
df_cluster_3

,index,Relative_Strength_90,Relative_Strength_120,Relative_Strength_150,Relative_Strength_180,volatility,Beta,Cluster
18,HDFCBANK.NS,-1.054595,0.156261,-0.639706,-0.634477,5.403550,0.594945,8
22,HINDUNILVR.NS,-1.545890,-0.553130,-0.856883,-0.931199,5.444496,0.139165,8
28,KOTAKBANK.NS,-1.634281,-0.312440,-0.922029,-0.984632,6.312671,0.717116,8
48,UPL.NS,-2.062566,-0.507480,-1.309056,-1.279761,6.020105,-0.084473,8


In [78]:
# Sort by 'Relative_Strength' in descending or
new_df.sort_values(by='Relative_Strength_90', ascending=False)

,index,Relative_Strength_90,Relative_Strength_120,Relative_Strength_150,Relative_Strength_180,volatility,Beta,Cluster
35,ONGC.NS,6.478071,3.389474,5.225217,4.997280,14.948841,2.206441,9
5,BAJAJ-AUTO.NS,6.007529,3.784307,7.098903,6.074200,17.728739,3.499750,9
42,TATAMOTORS.NS,5.485359,3.157849,5.025750,4.084616,13.852915,2.992754,1
9,BPCL.NS,4.996757,5.113720,6.853964,5.257258,29.606387,5.907420,3
8,BHARTIARTL.NS,4.590400,2.540316,4.030509,3.681586,7.570602,1.166507,6
12,COALINDIA.NS,4.513540,3.026497,6.434122,8.448006,23.299468,0.786089,2
39,SBIN.NS,4.260686,2.416897,2.981942,2.163480,12.183015,2.617167,1
36,POWERGRID.NS,3.927790,2.865240,4.806323,4.057732,12.185687,2.163626,1
43,TATASTEEL.NS,3.790301,2.161963,2.350257,2.565966,10.689530,0.927467,1
1,ADANIPORTS.NS,3.747011,3.913680,4.469270,5.207860,21.760110,4.269150,2
